# Домашнее задание 4
## Text Normalization 

deadline: 15 декабря 2019, 23:59

В этом домашнем задании вы будете работать с корпусом соревнования по нормализации текстов на русском языке. 

Ссылка на соревнование:
https://www.kaggle.com/c/text-normalization-challenge-russian-language

Корпус (train-test split) доступен там же, на kaggle. Кроме того, kaggle проверяет результаты на тестовом множестве. Пример сабмита в файле: ru_sample_submission_2. 

Задача заключается в том, привести исходный текст (колонку before) в нормализованную форму (колонка after). Дополнительно известны классы токенов (колонка class), общее число классов – 15. В тестовом множестве классы токенов отсутствуют. 

Корпус состоит из предложений на русском языке и их нормализованных аналогов. Примеры продемонстрированы на kaggle.

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 3-х человек.
2. Домашнее задание сдается через anytask, инвайты будут дополнительно высланы.
3. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо ipython-тетрадке. 
4. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
5. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
6. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
7. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 

In [1]:
DATA_PREFFIX = 'data/'
TEST_1_DATA_PATH = DATA_PREFFIX + 'ru_test.csv'
TEST_2_DATA_PATH = DATA_PREFFIX + 'ru_test_2.csv'
TRAIN_DATA_PATH = DATA_PREFFIX + 'ru_train.csv'
SIMPLE_DATA_TRAIN_PATH = DATA_PREFFIX + 'simple_data_train.tsv'
SIMPLE_DATA_TEST_PATH = DATA_PREFFIX + 'simple_data_test.tsv'

In [2]:
import pandas as pd
import numpy as np
import torch
import torchtext
import os
from tqdm import tqdm_notebook as tqdm
import pickle
import gc

In [3]:
MAX_DF = 10**3
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_train = df_train[:min(MAX_DF, df_train.shape[0])]

In [4]:
df_train[:10]

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,По,По
1,0,1,PLAIN,состоянию,состоянию
2,0,2,PLAIN,на,на
3,0,3,DATE,1862 год,тысяча восемьсот шестьдесят второй год
4,0,4,PUNCT,.,.
5,1,0,PLAIN,Оснащались,Оснащались
6,1,1,PLAIN,латными,латными
7,1,2,PLAIN,рукавицами,рукавицами
8,1,3,PLAIN,и,и
9,1,4,PLAIN,сабатонами,сабатонами


## Часть 1. [1 балл] Эксплоративный анализ

1. Найдите примеры каждого класса и опишите, по какой логике проведена нормализация токенов разных классов. 
2. В каких случаях токены класса PLAIN подвергаются нормализации? 
3. Напишите правила для нормализации токенов класса ORDINAL. 

#### Пункки 1.

In [5]:
classes = set(df_train['class'].values)
print(*classes)

ORDINAL LETTERS CARDINAL VERBATIM PUNCT MEASURE TELEPHONE PLAIN DATE


In [6]:
df_train[df_train['class'] == 'DIGIT'].head()

,sentence_id,token_id,class,before,after


* DIGIT состоит из цифр, переходит в последовательность слов 

In [7]:
df_train[df_train['class'] == 'MONEY'].head()

,sentence_id,token_id,class,before,after


* MONEY состоит из числа суммы денег и единицы измерения, переходит в соотв последовательность слов

In [8]:
df_train[df_train['class'] == 'LETTERS'].head()

,sentence_id,token_id,class,before,after
92,6,8,LETTERS,TV,t v
186,12,9,LETTERS,РСФСР,р с ф с р
301,24,1,LETTERS,СПб,с п б
340,26,1,LETTERS,СПб,с п б
346,26,7,LETTERS,В. А.,в а


* LETTERS состоит из аббревиатур, переходит в маленькие буквы, разделенные пробелами

In [9]:
df_train[df_train['class'] == 'CARDINAL'].head()

,sentence_id,token_id,class,before,after
137,9,9,CARDINAL,254,двести пятьдесят четыре
272,21,7,CARDINAL,2014,две тысячи четырнадцать
275,21,10,CARDINAL,12,двенадцать
304,24,4,CARDINAL,2014,две тысячи четырнадцать
343,26,4,CARDINAL,2011,две тысячи одиннадцать


* CARDINAL состоит из чисел, переходит в слова, обозначающие эти числа

In [10]:
df_train[df_train['class'] == 'FRACTION'].head()

,sentence_id,token_id,class,before,after


* FRACTION состоит из дробного числа, переходит в словарную запись дроби

In [11]:
df_train[df_train['class'] == 'TELEPHONE'].head()

,sentence_id,token_id,class,before,after
576,41,2,TELEPHONE,978-5-86007-700-3,девятьсот семьдесят восемь sil пять sil восемь...


* TELEPHONE состоит из номера телефона с дефисами, переходит в запись на русском языке, вместо дефисов в записи появляются слова 'sil'

In [12]:
df_train[df_train['class'] == 'DIGIT'].head()

,sentence_id,token_id,class,before,after


* DIGIT состоит из цифр, несущих в себе лишь смысл "последовательность цифр"

In [13]:
df_train[df_train['class'] == 'PUNCT'].head()

,sentence_id,token_id,class,before,after
4,0,4,PUNCT,.,.
14,1,9,PUNCT,.,.
18,2,3,PUNCT,",",","
24,2,9,PUNCT,(,(
27,2,12,PUNCT,),)


* PUNCT состоит из знаков пунктуации, переходит в себя

In [14]:
df_train[df_train['class'] == 'TIME'].head()

,sentence_id,token_id,class,before,after


* TIME состоит из времени, переходит в себя, когда точность до секунд, в последовательность слов, когда до минут

In [15]:
df_train[df_train['class'] == 'MEASURE'].head()

,sentence_id,token_id,class,before,after
365,27,12,MEASURE,61st,шестьдесят один стоун
412,30,13,MEASURE,31 минуту,тридцати одной минуту
474,34,12,MEASURE,500 км,пятисот километров
605,44,6,MEASURE,80 см.,восемьдесят сантиметров
793,55,18,MEASURE,480 с.,четыреста восемьдесят секунд


* MEASURE состоит из числа и единицы измерения, переходит в число словами и расшифровку этой единицы

In [16]:
df_train[df_train['class'] == 'VERBATIM'].head()

,sentence_id,token_id,class,before,after
60,4,4,VERBATIM,-,-
109,7,10,VERBATIM,兄,兄
110,7,11,VERBATIM,貴,貴
125,8,9,VERBATIM,-,-
212,16,7,VERBATIM,-,-


* VERBATIM состоит из специальных символов, переходит в себя

In [17]:
df_train[df_train['class'] == 'DECIMAL'].head()

,sentence_id,token_id,class,before,after


* DECIMAL состоит из десятичных дробей, переходит в слова

In [18]:
df_train[df_train['class'] == 'DATE'].head()

,sentence_id,token_id,class,before,after
3,0,3,DATE,1862 год,тысяча восемьсот шестьдесят второй год
17,2,2,DATE,1811 года,тысяча восемьсот одиннадцатого года
85,6,1,DATE,12 февраля 2013,двенадцатого февраля две тысячи тринадцатого года
90,6,6,DATE,15 февраля 2013,пятнадцатого февраля две тысячи тринадцатого года
189,13,1,DATE,1905 года,тысяча девятьсот пятого года


* DATE состоит из дат, переходит в слова

In [19]:
df_train[df_train['class'] == 'ELECTRONIC'].head()

,sentence_id,token_id,class,before,after


* ELECTRONIC состоит из аглийских слов и чисел. Англ слова переходят в их транскрипцию, числа в их запись словами

In [20]:
df_train[df_train['class'] == 'PLAIN'].head()

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,По,По
1,0,1,PLAIN,состоянию,состоянию
2,0,2,PLAIN,на,на
5,1,0,PLAIN,Оснащались,Оснащались
6,1,1,PLAIN,латными,латными


* PLAIN состоит из слов. Русские слова переходят в себя, англ слова в транскрипцию

In [21]:
df_train[df_train['class'] == 'ORDINAL'].head()

,sentence_id,token_id,class,before,after
53,3,17,ORDINAL,III,третьего
163,11,1,ORDINAL,1895,тысяча восемьсот девяносто пятом
164,11,2,ORDINAL,—1896,тысяча восемьсот девяносто шестом
485,35,5,ORDINAL,1796,тысяча семьсот девяносто шестого
487,35,7,ORDINAL,1802,тысяча восемьсот второй


* ORDINAL состоит из арабских или римских чисел, переходят в слова

Нормализация происходит так, чтобы Text-To-Speach системы могли произнести текст

#### Пункт 2.

In [22]:
MAX_SAMPLES = 10
samples_counter = 0
for i in range(df_train.shape[0]):
    cls = df_train['class'][i]
    before = df_train['before'][i]
    after = df_train['after'][i]
    if cls == 'PLAIN' and before != after:
        print('%-15s\t%-30s\t%-100s' % (cls, 'Before', 'After'))
        print('%-15s\t%-30s\t%-100s' % ('', df_train['before'][i], df_train['after'][i] ))
        print()
        samples_counter += 1
    if samples_counter == MAX_SAMPLES:
        break

PLAIN          	Before                        	After                                                                                               
               	Tiberius                      	т_trans и_trans б_trans е_trans р_trans и_trans у_trans с_trans                                     

PLAIN          	Before                        	After                                                                                               
               	Julius                        	д_trans ж_trans у_trans л_trans и_trans у_trans с_trans                                             

PLAIN          	Before                        	After                                                                                               
               	Pollienus                     	п_trans о_trans л_trans л_trans и_trans е_trans н_trans у_trans с_trans                             

PLAIN          	Before                        	After                                                         

Класс PLAIN преобразуется, если слово написано не на русском языке. Тогда оно преобразуется посредством транслитерации

#### Пункт 3.

In [23]:
MAX_SAMPLES = 50
samples_counter = 0
for i in range(df_train.shape[0]):
    cls = df_train['class'][i]
    before = df_train['before'][i]
    after = df_train['after'][i]
    if cls == 'ORDINAL' and before != after:
        print('%-15s\t%-30s\t%-100s' % (cls, 'Before', 'After'))
        print('%-15s\t%-30s\t%-100s' % ('', df_train['before'][i], df_train['after'][i] ))
        print()
        samples_counter += 1
    if samples_counter == MAX_SAMPLES:
        break

ORDINAL        	Before                        	After                                                                                               
               	III                           	третьего                                                                                            

ORDINAL        	Before                        	After                                                                                               
               	1895                          	тысяча восемьсот девяносто пятом                                                                    

ORDINAL        	Before                        	After                                                                                               
               	—1896                         	тысяча восемьсот девяносто шестом                                                                   

ORDINAL        	Before                        	After                                                         

*  Римские цифры переводятся в прилагательные (склонения мб зависят от контекста)

*  Когда после числа стоит -й/-я/-е/-х/-му, число переходит в прилагательное отвечающее на вопрос "который"/"которая"/"которые"/"в которых"/"которому"

*  Когда перед числом тире прилагательное отвечает на вопрос "в каком"

*  Когда число не окружено ничем оно переходит либо в прилагательное в зависящем от контекста склонении, либо остается числом. Скорее всего тоже зависит от контекста.

## Часть 2. [6 баллов]  seq2seq архитектуры
Имплементируйте несколько seq2seq архитектур. Энкодер получает на вход последовательность токенов before, декодер учится превращать их в токены after.
Энкодер и декодер работают на уровне символов, эмбеддинги символов инициализируются случайно (по аналогии с работами, в которых предложены нейросетевые модели исправления опечаток).

Эту часть задания рекомендуется выполнять с использованием allennlp (должно быть проще и удобнее).

1. [3 балла] LSTM encoder + LSTM decoder + три механизма внимания: скалярное произведение, аддитивное внимание и мультипликативное внимание (см. лекцию 6, слайд "подсчет весов attention")
2. [3 балла] Transformer

Используя автопровереку kaggle, оцените, как влияют параметры архитектуры на качество задачи.

[бонус] convolutional encoder + convolutional decoder

[бонус] pyramid LSTM (размер l+1 слоя в два раз меньше размера l, i-тый вход l+1 слоя – конкатенация выходов 2i и 2i+1)

In [24]:
from torch import nn
import itertools

import allennlp
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.instance import Instance

from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.iterators import BucketIterator

from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders.pytorch_seq2seq_wrapper import PytorchSeq2SeqWrapper
from allennlp.models.encoder_decoders import SimpleSeq2Seq

from allennlp.modules.attention.additive_attention import AdditiveAttention
from allennlp.modules.attention.bilinear_attention import BilinearAttention
from allennlp.modules.attention.dot_product_attention import DotProductAttention


from allennlp.training.trainer import Trainer

from allennlp.predictors import SimpleSeq2SeqPredictor

from sklearn.model_selection import train_test_split

In [25]:
ENCODER_HIDDEN_SIZE = 64
HIDDEN_DIM = 32
EMBEDDING_DIM = 256
MAX_DECODING_STEPS = 50
BATCH_SIZE=64
N_EPOCHS = 5

In [26]:
def prepare_simple_dataset(df, target_file_path):
    data = pd.DataFrame(df[['before', 'after']].values)
    data.to_csv(target_file_path, sep='\t', index=False)

df_train, df_test = train_test_split(df_train, test_size=0.3)
prepare_simple_dataset(df_train, SIMPLE_DATA_TRAIN_PATH)
prepare_simple_dataset(df_test, SIMPLE_DATA_TEST_PATH)

In [27]:
import builtins
import functools

old_open = open
uopen = functools.partial(open, encoding='utf8')
builtins.open = uopen

In [28]:
reader = Seq2SeqDatasetReader(
    source_tokenizer=CharacterTokenizer(),
    target_tokenizer=CharacterTokenizer(),
    source_token_indexers={'tokens': SingleIdTokenIndexer()},
    target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})

dataset_train = reader.read(SIMPLE_DATA_TRAIN_PATH)
dataset_test = reader.read(SIMPLE_DATA_TEST_PATH)

701it [00:00, 36993.04it/s]
301it [00:00, 27523.72it/s]


In [29]:
builtins.open = old_open

In [30]:
vocab = Vocabulary.from_instances(dataset_train,
                                      min_count={'tokens': 3, 'target_tokens': 3})

embeddings = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=EMBEDDING_DIM)

source_embedder = BasicTextFieldEmbedder({"tokens": embeddings})

100%|███████████████████████████████████████████████████████████████████████████████████████████| 701/701 [00:00<00:00, 100423.77it/s]


In [31]:
attentions = [None,
              DotProductAttention(),
              BilinearAttention(HIDDEN_DIM, HIDDEN_DIM),
              AdditiveAttention(HIDDEN_DIM, HIDDEN_DIM)
             ]

In [32]:
def make_model(attention_layer=None, embed_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM):    
    encoder = PytorchSeq2SeqWrapper(nn.LSTM(embed_dim, hidden_dim, batch_first=True))

    model = SimpleSeq2Seq(vocab, source_embedder, encoder, MAX_DECODING_STEPS,
                              target_embedding_dim=embed_dim,
                              target_namespace='target_tokens',
                              attention=attention_layer)
    return model

models = [make_model(att) for att in attentions]
optimizers = [torch.optim.Adam(model.parameters()) for model in models]

In [33]:
iterator = BucketIterator(batch_size=BATCH_SIZE, sorting_keys=[("source_tokens", "num_tokens")])
iterator.index_with(vocab)

trainers = [Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=dataset_train,
                  validation_dataset=dataset_test,
                  num_epochs=1,
                  cuda_device=0) for (model, optimizer) in zip(models, optimizers)]

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


In [49]:
from IPython.display import clear_output


def train(model, trainer, epochs=N_EPOCHS, reader=reader, clear_out=True):
    model.cuda()
    def tokens2text(tokens, start=0, end=None):
        tokens = list(tokens)
        end = len(tokens) if end is None else end
        tokens = tokens[start:end]
        text = map(str, tokens)
        text = ''.join(text)
        return text
    
    for i in range(epochs):
        predictor = SimpleSeq2SeqPredictor(model, reader)
        if clear_out:
            clear_output()
        trainer.train()

        predictor = SimpleSeq2SeqPredictor(model, reader)
        print('Epoch %d' % i)
        print('%-30s\t%-30s\t%-30s' % ('Before', 'True After', 'Pred After'))
        for instance in itertools.islice(dataset_test, 20):
            x_data = list(instance.fields['source_tokens'].tokens)
            y_true = instance.fields['target_tokens'].tokens
            y_pred = predictor.predict_instance(instance)['predicted_tokens']
            
            print('%-30s\t%-30s\t%-30s' % (tokens2text(x_data, 1, -1),
                                           tokens2text(y_true, 1, -1),
                                           tokens2text(y_pred)))
    model.cpu()

In [50]:
model_ind = 0

In [51]:
print('Training model with attention:', attentions[ind].__class__)
train(models[model_ind], trainers[model_ind])
gc.collect()
torch.cuda.empty_cache()

model_ind += 1

loss: 2.3148 ||: 100%|████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 14.15it/s]
BLEU: 0.0000, loss: 2.3816 ||: 100%|████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.38it/s]


Epoch 4
Before                        	True After                    	Pred After                    
0                             	1                             	.                             
по                            	по                            	в                             
.                             	.                             	.                             
Дик                           	Дик                           	.                             
.                             	.                             	.                             
и                             	и                             	в                             
они                           	они                           	и                             
успешно                       	успешно                       	и                             
конце                         	конце                         	и                             
перерастать                   	перерастать                   	

In [53]:
print('Training model with attention:', attentions[ind].__class__)
train(models[model_ind], trainers[model_ind])
model_ind += 1

loss: 2.9318 ||: 100%|████████████████████████████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  7.25it/s]
BLEU: 0.0000, loss: 2.9531 ||: 100%|████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  4.92it/s]


Epoch 4
Before                        	True After                    	Pred After                    
0                             	1                             	                              
по                            	по                            	                              
.                             	.                             	                              
Дик                           	Дик                           	                              
.                             	.                             	                              
и                             	и                             	                              
они                           	они                           	                              
успешно                       	успешно                       	                              
конце                         	конце                         	                              
перерастать                   	перерастать                   	

In [54]:
print('Training model with attention:', attentions[ind].__class__)
train(models[model_ind], trainers[model_ind])
model_ind += 1

loss: 2.9402 ||: 100%|████████████████████████████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  6.71it/s]
BLEU: 0.0004, loss: 2.9586 ||: 100%|████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  4.79it/s]


Epoch 4
Before                        	True After                    	Pred After                    
0                             	1                             	в                             
по                            	по                            	в                             
.                             	.                             	в                             
Дик                           	Дик                           	в                             
.                             	.                             	в                             
и                             	и                             	в                             
они                           	они                           	в                             
успешно                       	успешно                       	в                             
конце                         	конце                         	в                             
перерастать                   	перерастать                   	

In [55]:
print('Training model with attention:', attentions[ind].__class__)
train(models[model_ind], trainers[model_ind])
model_ind += 1

Training model with attention: <class 'allennlp.modules.attention.bilinear_attention.BilinearAttention'>


IndexError: list index out of range

## Часть 3. [2 балла]  Дополнительные признаки
Предложите и покажите, как можно было бы повысить качество нейросетевых моделей. Примерные варианты:
1. ансамблирование нейронных сетей
2. добавление морфологоческих признаков 
3. использование эмбеддингов слов 


## Часть 4. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Проведите анализ ошибок: когда модель ошибается? Можно ли скзаать, почему модель ошибается? Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?